# Prompt & QA Chain

Create a retrieval-augmented QA pipeline using:
- Custom prompt templates
- Your vector store (Chroma) and Hugging Face embeddings
- An LLM to generate answers based on retrieved remedy records

## Load the vector store

In [2]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)
vector_db = Chroma(
    persist_directory="avva_rag_vector_db",  
    embedding_function=embedding_model
)

retriever = vector_db.as_retriever(search_kwargs={"k": 3})


## Prompting Styles

| Style                      | Description                                                                | Common Use Cases                                    | Pros                                         | Cons                                                   |
| -------------------------- | -------------------------------------------------------------------------- | --------------------------------------------------- | -------------------------------------------- | ------------------------------------------------------ |
| **Zero-shot**              | Ask the model a task without any examples                                  | Simple Q\&A, classification                         | Fast, minimal effort                         | Lower accuracy on complex tasks                        |
| **One-shot**               | Provide one example to help steer the model                                | Tasks with subtle formatting needs                  | Slightly better than zero-shot               | Still limited generalization                           |
| **Few-shot**               | Show 2–5 examples before task prompt                                       | Code generation, sentiment analysis, math reasoning | Boosts accuracy with little supervision      | Sensitive to prompt length and token limits            |
| **Instruction-based**      | Provide clear, explicit instruction (e.g., “You are a helpful assistant…”) | Summarization, generation, reasoning                | Great for LLMs like GPT-3.5-turbo and Claude | Needs tuning, brittle with ambiguous instructions      |
| **Chain-of-thought (CoT)** | Ask model to explain reasoning step by step                                | Math problems, reasoning, logic puzzles             | Improves complex task accuracy               | Slower, may hallucinate if not well guided             |
| **Tree-of-thought (ToT)**  | Model explores multiple reasoning paths before choosing                    | Planning, multi-hop QA, games                       | Higher-quality decisions in reasoning tasks  | Requires external scaffolding or agents                |
| **Self-ask**               | Model asks itself intermediate questions                                   | Open-domain QA                                      | Encourages stepwise refinement               | May lose coherence if not filtered                     |
| **ReAct**                  | Reasoning + acting (choose a tool + reason + act recursively)              | Agents, tool-augmented systems                      | Interactive, tool-aware reasoning            | Requires tool integration (like LangChain Agents)      |
| **Reflexion**              | Model reflects on prior mistakes and corrects itself                       | Autonomous agents, iterative tasks                  | Self-correction loop improves outcomes       | Needs feedback loop and persistence                    |
| **RAG-style**              | Prompt includes retrieved context from vector DB                           | QA from documents, chatbots, LLM with memory        | Keeps answers grounded, boosts truthfulness  | Still prone to hallucinations if context is irrelevant |
| **Template-based**         | Prompts built from parameterized templates (LangChain-style)               | Programmatic prompting, pipelines                   | Modular and reusable                         | Might lack flexibility for nuance                      |


## Prompt Template: (Instructional + Context-Aware)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
You are Avva, a helpful and safety-aware AI assistant that suggests home remedies for common symptoms.

Use the information from the provided documents to answer the user's question. 
If no remedy is found, say: "I'm sorry, I couldn't find a trusted remedy for that symptom."

Always cite sources if available. Avoid giving medical advice beyond home remedies.

### User Question:
{question}

### Retrieved Context:
{context}

### Response:
""")


## Create QA Chain

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # looks for .env file
secrets_path = os.getenv("SECRETS_PATH")

In [6]:
import os
import json

with open(secrets_path, "r") as f:
    secrets = json.load(f)

import openai

openai.api_key = secrets["OPENAI_API_KEY"]

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=secrets["OPENAI_API_KEY"],
                 model="gpt-3.5-turbo", 
                 temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff", # injects context directly into the prompt
    chain_type_kwargs={"prompt": prompt_template}
)


In [9]:
question = "What are the best home remedies for sinus congestion?"
response = qa_chain.run(question)
print("🟩 Answer:", response)


/tmp/ipykernel_10505/2187608448.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)


🟩 Answer: Here are some safe home remedies for sinus congestion:

1. Aromatherapy: Using camphor or menthol salve around the nose can help break up mucus and reduce congestion. Breathing in essential oils like peppermint and eucalyptus can also be beneficial.

2. Nasal irrigation: Nasal irrigation therapy can help irrigate and hydrate nasal passages. Make sure to consult with your doctor before starting nasal irrigation therapy and use safe water sources like boiled and cooled tap water, sterile or distilled water, or saline solution.

3. Sleep with an extra pillow: Sleeping with an extra pillow under your head can help your sinuses drain.

(Source: Remedy documents)
